Este notebook tiene como objetivo procesar y preparar los datos para el desarrollo de un sistema de recomendación de películas, se trabajará en la limpieza del dataset credits.

Descripción del archivo credits.csv

El archivo credits.csv contiene información sobre el reparto y el equipo de producción de diversas películas. Se compone de las siguientes columnas:

    id: Identificador único de la película.

    cast: Lista en formato JSON con detalles de los actores, incluyendo su nombre, personaje interpretado, y otros datos.

    crew: Lista en formato JSON con detalles del equipo de producción, como el director, guionista y otros miembros clave.

⚙️ Proceso a realizar:
1️⃣ Cargar y visualizar la información del dataset.
2️⃣ Separar el DataFrame en dos archivos: uno con cast y otro con crew.
3️⃣ Reducir el tamaño eliminando el 40% de los registros.
se eliminaran estos registros por limitaciones de memoria 
4️⃣ Exportar los nuevos archivos en un formato más eficiente (Parquet).


In [25]:
import pandas as pd # type: ignore
import json
import os
import ast

In [5]:

# Cargar el archivo CSV
file_path = r"C:\Users\E B M\Documents\proyecto_mp\dataset_org\credits.csv"
credits_df = pd.read_csv(file_path)

In [6]:

#  Mostrar información general del DataFrame
print("\n Información general del DataFrame:")
print(credits_df.info())


 Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


In [13]:
#  Mostrar las primeras filas del DataFrame
print("\n Primeras 5 filas del DataFrame:")
print(credits_df.head())




 Primeras 5 filas del DataFrame:
                                                cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2  [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3  [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4  [{'cast_id': 1, 'character': 'George Banks', '...   

                                                crew     id  
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    862  
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   8844  
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...  15602  
3  [{'credit_id': '52fe44779251416c91011acb', 'de...  31357  
4  [{'credit_id': '52fe44959251416c75039ed7', 'de...  11862  


In [14]:
#  Verificar valores nulos
print("\n Valores nulos por columna:")
print(credits_df.isnull().sum())




 Valores nulos por columna:
cast    0
crew    0
id      0
dtype: int64


In [15]:
#  Tipos de datos de cada columna
print("\n Tipos de datos de cada columna:")
print(credits_df.dtypes)




 Tipos de datos de cada columna:
cast    object
crew    object
id       int64
dtype: object


In [16]:
#  Estadísticas generales (para columnas numéricas)
print("\n Estadísticas generales (columnas numéricas):")
print(credits_df.describe())




 Estadísticas generales (columnas numéricas):
                  id
count   45476.000000
mean   108345.997537
std    112443.796536
min         2.000000
25%     26443.250000
50%     60002.500000
75%    157302.000000
max    469172.000000


In [19]:
#  Visualizar algunas filas de las columnas 'cast' y 'crew'
print("\n Ejemplo de valores en la columna 'cast':")
print(credits_df['cast'].head(5))

print("\n Ejemplo de valores en la columna 'crew':")
print(credits_df['crew'].head(5))


 Ejemplo de valores en la columna 'cast':
0    [{'cast_id': 14, 'character': 'Woody (voice)',...
1    [{'cast_id': 1, 'character': 'Alan Parrish', '...
2    [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3    [{'cast_id': 1, 'character': "Savannah 'Vannah...
4    [{'cast_id': 1, 'character': 'George Banks', '...
Name: cast, dtype: object

 Ejemplo de valores en la columna 'crew':
0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
Name: crew, dtype: object


Vamos a crear dos DataFrames, uno para el reparto (cast) y otro para el equipo de producción (crew):

In [20]:
# Crear DataFrame para el reparto (cast)
cast_df = credits_df[['id', 'cast']].copy()

# Crear DataFrame para el equipo de producción (crew)
crew_df = credits_df[['id', 'crew']].copy()

# Mostrar información después de la separación
print("\n DataFrame de Cast:")
print(cast_df.info())

print("\n DataFrame de Crew:")
print(crew_df.info())



 DataFrame de Cast:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45476 non-null  int64 
 1   cast    45476 non-null  object
dtypes: int64(1), object(1)
memory usage: 710.7+ KB
None

 DataFrame de Crew:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45476 non-null  int64 
 1   crew    45476 non-null  object
dtypes: int64(1), object(1)
memory usage: 710.7+ KB
None


Reducir el tamaño el 40% de los registros Para optimizar la memoria

In [21]:
# Reducir el 40% de los registros de cada DataFrame
cast_df = cast_df.sample(frac=0.6, random_state=42).reset_index(drop=True)
crew_df = crew_df.sample(frac=0.6, random_state=42).reset_index(drop=True)

# Mostrar la cantidad de registros después de la reducción
print(f"\n🔹 Registros en cast después de la reducción: {cast_df.shape[0]}")
print(f"\n🔹 Registros en crew después de la reducción: {crew_df.shape[0]}")



🔹 Registros en cast después de la reducción: 27286

🔹 Registros en crew después de la reducción: 27286



luego desanidar los datos de los DataFrames crew_df y cast_df, que contienen información en formato de listas dentro de una columna. Primero, se identificará la estructura de estos datos para asegurarse de que sean listas de diccionarios. Luego, se usará explode() para separar cada elemento en filas individuales y pd.json_normalize() para extraer los atributos clave, como name, job y department en crew_df, y name, character y gender en cast_df. Finalmente, se añadirá la columna movie_id para mantener la relación con las películas, permitiendo un análisis más estructurado y eficiente.

In [27]:
# Asegurar que crew_df['crew'] es una lista de diccionarios
crew_df['crew'] = crew_df['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Expandir la lista de diccionarios
crew_exploded = crew_df.explode('crew')

# Convertir la columna 'crew' en un DataFrame con las claves desanidadas
crew_normalized = pd.json_normalize(crew_exploded['crew'])

# Ver los primeros registros
print(crew_normalized.head())


                  credit_id  department  gender       id                job  \
0  5630aa66c3a3681b5c00f74e  Production     2.0   7415.0            Casting   
1  52fe45c4c3a36847f80d9623     Writing     2.0  15244.0             Writer   
2  5630aac6c3a3681b5c00f759         Art     0.0  20777.0  Production Design   
3  5630a9f6c3a3681b4b00ea46  Production     2.0  43673.0           Producer   
4  5630aa0b9251414ab700d927       Sound     2.0  38309.0              Music   

               name                      profile_path  
0   Rick Montgomery                              None  
1      Ehren Kruger  /3aC4aA5DKv6fymluT2X6UQ1iKua.jpg  
2  Humphrey Bangham                              None  
3         Tom Reeve                              None  
4     Gast Waltzing                              None  


In [28]:
directores_df = crew_normalized[crew_normalized['job'] == "Director"]
print(directores_df[['id', 'name']])  # Muestra ID y nombre del director


               id                name
6        179616.0      Richard Spence
14       190103.0    Alain Desrochers
79         2000.0     Lewis Milestone
108       32375.0     Mario Monicelli
114       20556.0  Bertrand Tavernier
...           ...                 ...
281172   583745.0       Antoni Krauze
281175   100793.0    Reginald Le Borg
281179   114997.0         Ángel Muñiz
281182    19093.0     George Marshall
281188  1162038.0     Lawrie Brewster

[29464 rows x 2 columns]


In [29]:
# Asegurar que cast_df['cast'] es una lista de diccionarios
cast_df['cast'] = cast_df['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Expandir la lista de diccionarios
cast_exploded = cast_df.explode('cast')

# Convertir la columna 'cast' en un DataFrame con las claves desanidadas
cast_normalized = pd.json_normalize(cast_exploded['cast'])

# Ver los primeros registros
print(cast_normalized.head())

   cast_id               character                 credit_id  gender  \
0      4.0              David Marx  52fe45c4c3a36847f80d9627     2.0   
1      5.0             Kurt Bishop  52fe45c4c3a36847f80d962b     2.0   
2      6.0            Kris Paddock  52fe45c4c3a36847f80d962f     1.0   
3      7.0               Mark Ohai  52fe45c4c3a36847f80d9633     0.0   
4      1.0  Eduardo 'Eddie' Deacon  59379a76c3a36827dd0009be     2.0   

         id              name  order                      profile_path  
0     585.0      Rutger Hauer    0.0  /2x1S2VAUvZXZuDjZ4E9iEKINvNu.jpg  
1   37041.0   Andrew McCarthy    1.0  /bn6djJHhLF6dTIz5H3UvuCFoeLm.jpg  
2   47615.0   Tara Fitzgerald    2.0  /qpS1nff6LgwbWXzS14qsmW6YIOi.jpg  
3  202939.0      Hari Dhillon    3.0  /mzlWa84jhQNjvgHAqtnMxl03yUK.jpg  
4    3131.0  Antonio Banderas    0.0  /85197jARsr06xQ84NhP9YoBL3sR.jpg  


In [30]:
cast_final = cast_normalized[['id', 'name', 'character', 'gender']]
print(cast_final.head())


         id              name               character  gender
0     585.0      Rutger Hauer              David Marx     2.0
1   37041.0   Andrew McCarthy             Kurt Bishop     2.0
2   47615.0   Tara Fitzgerald            Kris Paddock     1.0
3  202939.0      Hari Dhillon               Mark Ohai     0.0
4    3131.0  Antonio Banderas  Eduardo 'Eddie' Deacon     2.0


Y finalmente, voy a exportar los DataFrames cast_df y crew_df en formato Parquet para optimizar su almacenamiento y procesamiento. Me aseguro de que la carpeta de destino dataset_limpio exista, y luego guardo cada DataFrame en su respectivo archivo (cast.parquet y crew.parquet). De esta manera, los datos quedan organizados y listos para ser utilizados en futuras consultas o análisis.

In [31]:

# Ruta de exportación
export_path = r"C:\Users\E B M\Documents\proyecto_mp\dataset_limpio"
os.makedirs(export_path, exist_ok=True)

# Definir rutas de los archivos
cast_file = os.path.join(export_path, "cast.parquet")
crew_file = os.path.join(export_path, "crew.parquet")

# Guardar en formato Parquet los DataFrames procesados
cast_normalized.to_parquet(cast_file, index=False)
crew_normalized.to_parquet(crew_file, index=False)

print(f"Archivos exportados en: {export_path}")
print(f" - Cast: {cast_file}")
print(f" - Crew: {crew_file}")


Archivos exportados en: C:\Users\E B M\Documents\proyecto_mp\dataset_limpio
 - Cast: C:\Users\E B M\Documents\proyecto_mp\dataset_limpio\cast.parquet
 - Crew: C:\Users\E B M\Documents\proyecto_mp\dataset_limpio\crew.parquet
